In [1]:
#Installation der notwendigen Bibliotheken
!pip install -U langchain langchain-community langchain-ollama faiss-cpu tiktoken PyPDF2 pypdf ollama

     ---------------------------------------- 0.0/170.8 kB ? eta -:--:--
     --------------------------- ---------- 122.9/170.8 kB 3.6 MB/s eta 0:00:01
     -------------------------------------- 170.8/170.8 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/42.9 kB ? eta -:--:--
     -------------------------------------- - 41.0/42.9 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- - 41.0/42.9 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 42.9/42.9 kB 299.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.0 MB 3.2 MB/s eta 0:00:01
   ------------- -------------------------- 0.3/1.0 MB 4.2 MB/s eta 0:00:01
   --------------------- ------------------ 0.5/1.0 MB 3.8 MB/s eta 0:00:01
   ---------------------------- ----------- 0.7/1.0 MB 3.7 MB/s eta 0:00:01
   ----------------------------------- ---- 0.9/1.0 MB 3.7 MB/s eta 0:00:01
   ---

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.2 which is incompatible.


In [3]:
# Bibliotheken importieren
import os
import ollama
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from pydantic import Field
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Schritt 1: OllamaLLM-Klasse erstellen (benutzerdefinierte Implementierung)
class OllamaLLM(LLM):
    model_name: str = Field(default="llama3.2")

    def __init__(self, model_name: str):
        super().__init__()
        self.model_name = model_name

    def _call(self, prompt: str, stop = None) -> str:
        response = ollama.chat(model=self.model_name, messages=[{"role": "user", "content": prompt}])
        return response['message']['content']

    @property
    def _llm_type(self) -> str:
        return "ollama"

# Schritt 2: Dokumente laden und vorbereiten
pdf_loader = PyPDFLoader("RAG_4_LLMs.pdf")
pdf_documents = loader.load()
text_loader = TextLoader("testdata.txt")
text_documents = text_loader.load()
documents = pdf_documents + text_documents

# Schritt 3: Text in Chunks aufteilen
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(documents)

# Schritt 4: Embeddings generieren (OpenAI-Embeddings, da diese nach wie vor benötigt werden)
embeddings = OllamaEmbeddings(model="llama3.2")
# Modell muss mittels Ollama lokal installiert sein (ollama pull llama3.2)

# Schritt 5: Vektorindex erstellen
vector_store = FAISS.from_documents(chunks, embeddings)

# Schritt 6: Retrieval und LLM-Kette erstellen mit OllamaLLM
llm = OllamaLLM(model_name="llama3.2")
# Modell muss mittels Ollama lokal installiert sein (ollama pull llama3.2)
retriever=vector_store.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

# Schritt 7: Frage stellen
query1 = "Worum geht es in dem Paper Retrieval-Augmented Generation for Large Language Models?"
response1 = qa_chain.invoke(query1)
query2 = "Wer ist der Studiengangsleiter der Informationstechnik?"
response2 = qa_chain.invoke(query2)
query3 = "Wer ist Rolf Assfalg?"
response3 = qa_chain.invoke(query3)

# Antwort ausgeben
print("Antwort:", response1)
print("Antwort:", response2)
print("Antwort:", response3)


NameError: name 'loader' is not defined